In [1]:
!pip install -q tensorflow-recommenders
!pip install -q fastparquet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx-bsl 1.9.0 requires pyarrow<6,>=1, but you have pyarrow 8.0.0 which is incompatible.
tensorflow-transform 1.9.0 requires pyarrow<6,>=1, but you have pyarrow 8.0.0 which is incompatible.
tensorflow-transform 1.9.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,!=2.8.*,<2.10,>=1.15.5, but you have tensorflow 2.11.0 which is incompatible.
tensorflow-io 0.21.0 requires tensorflow<2.7.0,>=2.6.0, but you have tensorflow 2.11.0 which is incompatible.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, but you have tensorflow-io-gcs-filesystem 0.30.0 which is incompatible.


In [2]:
!pip install -q tensorflow

In [3]:
# Import packages
import gc
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
from tqdm import tqdm

from typing import Dict, Text


# Import tensorflow package
import tensorflow as tf
print(f'Running tensorflow version {tf.__version__}')

# Import tensorflow-recommenders package
import tensorflow_recommenders as tfrs

2023-01-31 14:44:55.306376: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-31 14:44:55.486101: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:
2023-01-31 14:44:55.486177: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-31 14:44:57.131043: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

Running tensorflow version 2.11.0


In [4]:
_BATCHSIZE = 16384
_TEST_BATCHSIZE = _BATCHSIZE // 2
_BASE_DIR = '/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/'
_TEST_DIR = '/kaggle/input/otto-chunk-data-inparquet-format/test_parquet/'
_EPOCHS = 1

In [5]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object and col_type.name != 'category' and 'datetime' not in col_type.name:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        elif 'datetime' not in col_type.name:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [6]:
base_chunks = [#'/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/010000000_010100000.parquet', 
          #'/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/010100000_010200000.parquet', 
          #'/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/010200000_010300000.parquet', 
          #'/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/010300000_010400000.parquet',
         #'/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/010400000_010500000.parquet', 
         #'/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/010500000_010600000.parquet', 
          #'/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/010600000_010700000.parquet', 
          #'/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/010700000_010800000.parquet',
         #'/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/010800000_010900000.parquet', 
          #'/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/010900000_011000000.parquet', 
          #'/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/011000000_011100000.parquet', 
          #'/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/011100000_011200000.parquet',
         #'/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/011200000_011300000.parquet', 
          #'/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/011300000_011400000.parquet', 
          #'/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/011400000_011500000.parquet', 
          #'/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/011500000_011600000.parquet',
         #'/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/011600000_011700000.parquet', 
          #'/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/011700000_011800000.parquet', 
          #'/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/011800000_011900000.parquet', 
          '/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/011900000_012000000.parquet',
         '/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/012000000_012100000.parquet', 
          '/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/012100000_012200000.parquet', 
          '/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/012200000_012300000.parquet', 
          '/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/012300000_012400000.parquet',
         '/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/012400000_012500000.parquet', 
          '/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/012500000_012600000.parquet', 
          '/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/012600000_012700000.parquet', 
          '/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/012700000_012800000.parquet',
         '/kaggle/input/otto-chunk-data-inparquet-format/train_parquet/012800000_012900000.parquet']
additional_chunks = glob(_TEST_DIR + '*.parquet')
chunks = base_chunks + additional_chunks  

In [7]:
df_f = (pd
      .read_parquet(glob(_BASE_DIR + '*.parquet'), engine='fastparquet')
      .pipe(reduce_mem_usage)
      .sort_values(by=['session'], ascending=True)
      .reset_index(drop=True)
     )

article_vocab = list(df_f['aid'].unique())
session_vocab = list(df_f['session'].unique())
type_vocab = ['clicks', 'carts', 'orders']

del df_f
gc.collect()

"df_f = (pd\n      .read_parquet(chunks, engine='fastparquet')\n      .pipe(reduce_mem_usage)\n      .sort_values(by=['session'], ascending=True)\n      .reset_index(drop=True)\n     )\n\narticle_vocab = list(df_f['aid'].unique())\nsession_vocab = list(df_f['session'].unique())\ntype_vocab = ['clicks', 'carts', 'orders']\n\ndel df_f\ngc.collect()"

In [8]:
df = (pd.read_parquet(chunks, engine='fastparquet')
          .pipe(reduce_mem_usage))
display(df.head(n=10))
display(df.info())

Memory usage of dataframe is 30.02 MB
Memory usage after optimization is: 15.95 MB
Decreased by 46.9%


,session,aid,ts,type
0,10000000,938151,1660838216020,clicks
1,10000000,938151,1660839335573,clicks
2,10000001,921338,1660838216273,carts
3,10000001,921338,1660838278053,clicks
4,10000001,921338,1660838308474,clicks
5,10000001,921338,1660838329660,carts
6,10000002,1625942,1660838216523,clicks
7,10000002,1196256,1661544246289,clicks
8,10000002,1196256,1661544623231,clicks
9,10000002,1196256,1661544715884,clicks


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 983825 entries, 0 to 983824
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype   
---  ------   --------------   -----   
 0   session  983825 non-null  int32   
 1   aid      983825 non-null  int32   
 2   ts       983825 non-null  int64   
 3   type     983825 non-null  category
dtypes: category(1), int32(2), int64(1)
memory usage: 16.0 MB


None

In [9]:
#article_vocab = list(df['aid'].unique())
#session_vocab = list(df['session'].unique())
#type_vocab = ['clicks', 'carts', 'orders']
#df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 983825 entries, 0 to 983824
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype   
---  ------   --------------   -----   
 0   session  983825 non-null  int32   
 1   aid      983825 non-null  int32   
 2   ts       983825 non-null  int64   
 3   type     983825 non-null  category
dtypes: category(1), int32(2), int64(1)
memory usage: 16.0 MB


In [10]:
tensor = tf.data.Dataset.from_tensor_slices(dict(df))
tensor = tf.data.Dataset.prefetch(tensor, buffer_size=tf.data.AUTOTUNE)

features = tensor.map(lambda x: {
    "session": x["session"],
    "type": x["type"],
    "aid": x["aid"]
})


unique_aids_df = pd.DataFrame(df['aid'].unique(), columns=['aid'])
targets = tf.data.Dataset.from_tensor_slices(dict(unique_aids_df))
targets = tf.data.Dataset.prefetch(targets, buffer_size=tf.data.AUTOTUNE)

targets = targets.map(lambda x: x["aid"])

2023-01-31 14:45:01.964613: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:
2023-01-31 14:45:01.964669: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-31 14:45:01.964699: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (5ce04aa06b7b): /proc/driver/nvidia/version does not exist
2023-01-31 14:45:01.965288: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other 

In [11]:
del df
gc.collect()

75

In [12]:
tf.random.set_seed(42)

#train = features.take(80_000)
#test = features.skip(80_000).take(20_000)

cached_train = features.cache().batch(_BATCHSIZE)
#cached_test = test.cache().batch(_TEST_BATCHSIZE)

In [13]:
class MultiObjectRetrieval(tfrs.Model):

    def __init__(self, article_model, session_model, type_model):
        
        super().__init__()
        
        self.article_model: tf.keras.Model = article_model
        self.session_model: tf.keras.Model = session_model
        self.type_model: tf.keras.Model = type_model   
        self.feat_model = tf.keras.Sequential([
                    tf.keras.layers.Dense(16),
                ])            
            
        self.task: tf.keras.layers.Layer = task


    def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
        
        session_embeddings = self.session_model(features['session'])
        article_embeddings = self.article_model(features['aid'])
        type_embeddings = self.type_model(features['type'])
        
        return (
            article_embeddings, 
                 self.feat_model(
                tf.concat([session_embeddings, type_embeddings], axis=1)
            )
        )

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        
        article_embeddings, feature_embeddings = self(features)
        
        loss = self.task(feature_embeddings, article_embeddings)


        return loss

In [14]:
TPU = False
if TPU:
    print("Use TPU...")
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
elif tf.config.list_physical_devices('GPU'):
    print("Use GPU devices...")
    strategy = (tf
                .distribute
                .MirroredStrategy(
                    cross_device_ops = tf.distribute.HierarchicalCopyAllReduce() 
                                 )
               )
elif tf.config.list_physical_devices('CPU'):  
    print("No GPUs/ TPUs available. Using default strategy...")
    strategy = tf.distribute.get_strategy()

No GPUs/ TPUs available. Using default strategy...


In [15]:
%%time
with strategy.scope():
    
    article_model = tf.keras.Sequential(
        [tf.keras.layers.IntegerLookup(vocabulary=article_vocab),      
            tf.keras.layers.Embedding(len(article_vocab)+1, 16)]
    )     

    session_model = tf.keras.Sequential(
        [tf.keras.layers.IntegerLookup(vocabulary=session_vocab),     
            tf.keras.layers.Embedding(len(session_vocab)+1, 8)]
    )   

    type_model = tf.keras.Sequential(
        [tf.keras.layers.StringLookup(vocabulary=type_vocab),     
            tf.keras.layers.Embedding(len(type_vocab)+1, 8)]
    )  

    candidates = (targets.batch(128).map(article_model)
    )

    metrics = tfrs.metrics.FactorizedTopK(
        candidates = candidates
    )

    task = tfrs.tasks.Retrieval(
      metrics = metrics
    )

    
    del tensor, train, test
    gc.collect()
    
    model = MultiObjectRetrieval(article_model, session_model, type_model)
    model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

CPU times: user 10 s, sys: 150 ms, total: 10.2 s
Wall time: 10.1 s


In [16]:
model.fit(cached_train, epochs=_EPOCHS)

5/5 [==============================] - 835s 163s/step - factorized_top_k/top_1_categorical_accuracy: 7.5000e-05 - factorized_top_k/top_5_categorical_accuracy: 2.6250e-04 - factorized_top_k/top_10_categorical_accuracy: 3.5000e-04 - factorized_top_k/top_50_categorical_accuracy: 0.0012 - factorized_top_k/top_100_categorical_accuracy: 0.0020 - loss: 152178.3594 - regularization_loss: 0.0000e+00 - total_loss: 152178.3594


In [17]:
#model.evaluate(cached_test, return_dict=True)

In [18]:
sample_pred = {"session": np.array([12899779]),
      "ts": np.array([1661724000278]),
      "type": np.array(['clicks']),
      "aid": np.array([59625])
      }

session_embeddings = model.session_model(sample_pred['session'])
type_embeddings = model.type_model(sample_pred['type'])

index = tfrs.layers.factorized_top_k.BruteForce(model.feat_model)
index.index_from_dataset(
  tf.data.Dataset.zip((targets.batch(100), targets.batch(100).map(model.article_model)))
)

_, titles = index(tf.concat([session_embeddings, type_embeddings], axis=1))
print(f"Recommendations for session {12899779}: {titles[:20]}")

Recommendations for session 12899779: [[ 753525  979863 1796103  271231  199409 1255910 1731920 1242595  108125
  1673641]]


In [19]:
from tqdm import tqdm

index = tfrs.layers.factorized_top_k.BruteForce(model.feat_model)
index.index_from_dataset(
  tf.data.Dataset.zip((targets.batch(100), targets.batch(100).map(model.article_model)))
)
test = pd.read_parquet(additional_chunks, engine='fastparquet')
test_session_AIDs = test.reset_index(drop=True).groupby('session')['aid'].apply(list)
test_session_types = test.reset_index(drop=True).groupby('session')['type'].apply(list)
types = ['clicks', 'carts', 'orders']
del test
gc.collect()

preds_dict = {}

sessiontypes_list = [str(session) + '_' + type for session in test_session_types.index for type in types]
sessiontypes_list = [[int(st.split('_')[0]), st.split('_')[1]] for st in sessiontypes_list]
sessions = [session[0] for session in sessiontypes_list]
sessions_embeddings = model.session_model(np.array(sessions))
types = [type[1] for type in sessiontypes_list]
types_embeddings = model.type_model(np.array(types))

for i in tqdm(range(0,len(sessions_embeddings))):
    _, titles = index(tf.concat([[sessions_embeddings[i]], [types_embeddings[i]]], axis=1))
    preds_dict[str(sessions[i]) + '_' + types[i]] = titles[:20].numpy()

  1%|          | 51912/5015409 [06:15<9:58:50, 138.14it/s] 


KeyboardInterrupt: 

In [20]:
submission = pd.DataFrame({'session_type': preds_dict.keys(), 'labels': [list(i[0]) for i in list(preds_dict.values())]})

submission.to_csv('submission.csv', index=False)

In [ ]:
!mkdir -p saved_model
tf.saved_model.save(index, '/kaggle/working/saved_model')

In [ ]:
#loaded = tf.saved_model.load(path)
#scores, titles = loaded(["42"])